In [15]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from tqdm import tqdm

from transformers import BertTokenizer, BertModel, TFAutoModel, AutoTokenizer
import torch

pd.set_option('display.max_column', None)

In [2]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

df_train.drop('id', axis=1, inplace=True)
df_test.drop('id', axis=1, inplace=True)

In [3]:
df_train.head()

,text,author
0,"This process, however, afforded me no means of...",EAP
1,It never once occurred to me that the fumbling...,HPL
2,"In his left hand was a gold snuff box, from wh...",EAP
3,How lovely is spring As we looked from Windsor...,MWS
4,"Finding nothing else, not even gold, the Super...",HPL


In [4]:
df_test.head()

,text
0,"Still, as I urged our leaving Ireland with suc..."
1,"If a fire wanted fanning, it could readily be ..."
2,And when they had broken down the frail door t...
3,While I was thinking how I should possibly man...
4,I am not sure to what limit his knowledge may ...


In [5]:
for col in df_train.columns:
    print(f'The number of unique values in {col} is {df_train[col].nunique()}')

The number of unique values in text is 19579
The number of unique values in author is 3


In [6]:
df_train[df_train.duplicated()]

,text,author


In [7]:
df_test[df_test.duplicated()]

,text


In [8]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19579 entries, 0 to 19578
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    19579 non-null  object
 1   author  19579 non-null  object
dtypes: object(2)
memory usage: 306.0+ KB


# Preprocess the text
    - Lowercase
    - Pytorch BERT
    - TensorFlow BERT

In [9]:
documents_train = df_train['text'].tolist()
documents_test = df_test['text'].tolist()

### Pytorch BERT

In [10]:
%%time

tokenizer = BertTokenizer.from_pretrained('google-bert/bert-large-uncased')
model = BertModel.from_pretrained('google-bert/bert-large-uncased')

CPU times: total: 922 ms
Wall time: 32.1 s


In [11]:
def get_bert_embeddings(sentences):
    model.eval()  # Put the model in evaluation mode
    batch_size = 64  # Adjust based on your memory availability
    embeddings = []
    
    # Wrap the range generator with tqdm for a progress bar
    for i in tqdm(range(0, len(sentences), batch_size), desc="Processing batches"):
        batch = sentences[i:i+batch_size]
        inputs = tokenizer(batch, padding=True, truncation=True, return_tensors="pt")
        with torch.no_grad():
            outputs = model(**inputs)
        embeddings.append(outputs.last_hidden_state[:, 0, :].detach().numpy())
    
    # Concatenate all batch embeddings
    embeddings = np.concatenate(embeddings, axis=0)
    return embeddings

In [12]:
%%time

bert_embeddings = get_bert_embeddings(documents_train)
bert_df_train = pd.DataFrame(bert_embeddings)
bert_df_train.columns = ['bert_' + str(col) for col in bert_df_train.columns]
bert_df_train.head()

Processing batches: 100%|██████████| 306/306 [5:23:28<00:00, 63.43s/it]     


CPU times: total: 6h 10min 35s
Wall time: 5h 23min 33s


,bert_0,bert_1,bert_2,bert_3,bert_4,bert_5,bert_6,bert_7,bert_8,bert_9,bert_10,bert_11,bert_12,bert_13,bert_14,bert_15,bert_16,bert_17,bert_18,bert_19,bert_20,bert_21,bert_22,bert_23,bert_24,bert_25,bert_26,bert_27,bert_28,bert_29,bert_30,bert_31,bert_32,bert_33,bert_34,bert_35,bert_36,bert_37,bert_38,bert_39,bert_40,bert_41,bert_42,bert_43,bert_44,bert_45,bert_46,bert_47,bert_48,bert_49,bert_50,bert_51,bert_52,bert_53,bert_54,bert_55,bert_56,bert_57,bert_58,bert_59,bert_60,bert_61,bert_62,bert_63,bert_64,bert_65,bert_66,bert_67,bert_68,bert_69,bert_70,bert_71,bert_72,bert_73,bert_74,bert_75,bert_76,bert_77,bert_78,bert_79,bert_80,bert_81,bert_82,bert_83,bert_84,bert_85,bert_86,bert_87,bert_88,bert_89,bert_90,bert_91,bert_92,bert_93,bert_94,bert_95,bert_96,bert_97,bert_98,bert_99,bert_100,bert_101,bert_102,bert_103,bert_104,bert_105,bert_106,bert_107,bert_108,bert_109,bert_110,bert_111,bert_112,bert_113,bert_114,bert_115,bert_116,bert_117,bert_118,bert_119,bert_120,bert_121,bert_122,bert_123,bert_124,bert_125,bert_126,bert_127,bert_128,bert_129,bert_130,bert_131,bert_132,bert_133,bert_134,bert_135,bert_136,bert_137,bert_138,bert_139,bert_140,bert_141,bert_142,bert_143,bert_144,bert_145,bert_146,bert_147,bert_148,bert_149,bert_150,bert_151,bert_152,bert_153,bert_154,bert_155,bert_156,bert_157,bert_158,bert_159,bert_160,bert_161,bert_162,bert_163,bert_164,bert_165,bert_166,bert_167,bert_168,bert_169,bert_170,bert_171,bert_172,bert_173,bert_174,bert_175,bert_176,bert_177,bert_178,bert_179,bert_180,bert_181,bert_182,bert_183,bert_184,bert_185,bert_186,bert_187,bert_188,bert_189,bert_190,bert_191,bert_192,bert_193,bert_194,bert_195,bert_196,bert_197,bert_198,bert_199,bert_200,bert_201,bert_202,bert_203,bert_204,bert_205,bert_206,bert_207,bert_208,bert_209,bert_210,bert_211,bert_212,bert_213,bert_214,bert_215,bert_216,bert_217,bert_218,bert_219,bert_220,bert_221,bert_222,bert_223,bert_224,bert_225,bert_226,bert_227,bert_228,bert_229,bert_230,bert_231,bert_232,bert_233,bert_234,bert_235,bert_236,bert_237,bert_238,bert_239,bert_240,bert_241,bert_242,bert_243,bert_244,bert_245,bert_246,bert_247,bert_248,bert_249,bert_250,bert_251,bert_252,bert_253,bert_254,bert_255,bert_256,bert_257,bert_258,bert_259,bert_260,bert_261,bert_262,bert_263,bert_264,bert_265,bert_266,bert_267,bert_268,bert_269,bert_270,bert_271,bert_272,bert_273,bert_274,bert_275,bert_276,bert_277,bert_278,bert_279,bert_280,bert_281,bert_282,bert_283,bert_284,bert_285,bert_286,bert_287,bert_288,bert_289,bert_290,bert_291,bert_292,bert_293,bert_294,bert_295,bert_296,bert_297,bert_298,bert_299,bert_300,bert_301,bert_302,bert_303,bert_304,bert_305,bert_306,bert_307,bert_308,bert_309,bert_310,bert_311,bert_312,bert_313,bert_314,bert_315,bert_316,bert_317,bert_318,bert_319,bert_320,bert_321,bert_322,bert_323,bert_324,bert_325,bert_326,bert_327,bert_328,bert_329,bert_330,bert_331,bert_332,bert_333,bert_334,bert_335,bert_336,bert_337,bert_338,bert_339,bert_340,bert_341,bert_342,bert_343,bert_344,bert_345,bert_346,bert_347,bert_348,bert_349,bert_350,bert_351,bert_352,bert_353,bert_354,bert_355,bert_356,bert_357,bert_358,bert_359,bert_360,bert_361,bert_362,bert_363,bert_364,bert_365,bert_366,bert_367,bert_368,bert_369,bert_370,bert_371,bert_372,bert_373,bert_374,bert_375,bert_376,bert_377,bert_378,bert_379,bert_380,bert_381,bert_382,bert_383,bert_384,bert_385,bert_386,bert_387,bert_388,bert_389,bert_390,bert_391,bert_392,bert_393,bert_394,bert_395,bert_396,bert_397,bert_398,bert_399,bert_400,bert_401,bert_402,bert_403,bert_404,bert_405,bert_406,bert_407,bert_408,bert_409,bert_410,bert_411,bert_412,bert_413,bert_414,bert_415,bert_416,bert_417,bert_418,bert_419,bert_420,bert_421,bert_422,bert_423,bert_424,bert_425,bert_426,bert_427,bert_428,bert_429,bert_430,bert_431,bert_432,bert_433,bert_434,bert_435,bert_436,bert_437,bert_438,bert_439,bert_440,bert_441,bert_442,bert_443,bert_444,bert_445,bert_446,bert_447,bert_448,bert_449,bert_450,bert_451,bert_452,bert_453,bert_454,bert_455,bert_

In [13]:
%%time

bert_embeddings = get_bert_embeddings(documents_test)
bert_df_test = pd.DataFrame(bert_embeddings)
bert_df_test.columns = ['bert_' + str(col) for col in bert_df_test.columns]
bert_df_test.head()

Processing batches: 100%|██████████| 132/132 [1:40:22<00:00, 45.63s/it]   


CPU times: total: 2h 41min 48s
Wall time: 1h 40min 23s


,bert_0,bert_1,bert_2,bert_3,bert_4,bert_5,bert_6,bert_7,bert_8,bert_9,bert_10,bert_11,bert_12,bert_13,bert_14,bert_15,bert_16,bert_17,bert_18,bert_19,bert_20,bert_21,bert_22,bert_23,bert_24,bert_25,bert_26,bert_27,bert_28,bert_29,bert_30,bert_31,bert_32,bert_33,bert_34,bert_35,bert_36,bert_37,bert_38,bert_39,bert_40,bert_41,bert_42,bert_43,bert_44,bert_45,bert_46,bert_47,bert_48,bert_49,bert_50,bert_51,bert_52,bert_53,bert_54,bert_55,bert_56,bert_57,bert_58,bert_59,bert_60,bert_61,bert_62,bert_63,bert_64,bert_65,bert_66,bert_67,bert_68,bert_69,bert_70,bert_71,bert_72,bert_73,bert_74,bert_75,bert_76,bert_77,bert_78,bert_79,bert_80,bert_81,bert_82,bert_83,bert_84,bert_85,bert_86,bert_87,bert_88,bert_89,bert_90,bert_91,bert_92,bert_93,bert_94,bert_95,bert_96,bert_97,bert_98,bert_99,bert_100,bert_101,bert_102,bert_103,bert_104,bert_105,bert_106,bert_107,bert_108,bert_109,bert_110,bert_111,bert_112,bert_113,bert_114,bert_115,bert_116,bert_117,bert_118,bert_119,bert_120,bert_121,bert_122,bert_123,bert_124,bert_125,bert_126,bert_127,bert_128,bert_129,bert_130,bert_131,bert_132,bert_133,bert_134,bert_135,bert_136,bert_137,bert_138,bert_139,bert_140,bert_141,bert_142,bert_143,bert_144,bert_145,bert_146,bert_147,bert_148,bert_149,bert_150,bert_151,bert_152,bert_153,bert_154,bert_155,bert_156,bert_157,bert_158,bert_159,bert_160,bert_161,bert_162,bert_163,bert_164,bert_165,bert_166,bert_167,bert_168,bert_169,bert_170,bert_171,bert_172,bert_173,bert_174,bert_175,bert_176,bert_177,bert_178,bert_179,bert_180,bert_181,bert_182,bert_183,bert_184,bert_185,bert_186,bert_187,bert_188,bert_189,bert_190,bert_191,bert_192,bert_193,bert_194,bert_195,bert_196,bert_197,bert_198,bert_199,bert_200,bert_201,bert_202,bert_203,bert_204,bert_205,bert_206,bert_207,bert_208,bert_209,bert_210,bert_211,bert_212,bert_213,bert_214,bert_215,bert_216,bert_217,bert_218,bert_219,bert_220,bert_221,bert_222,bert_223,bert_224,bert_225,bert_226,bert_227,bert_228,bert_229,bert_230,bert_231,bert_232,bert_233,bert_234,bert_235,bert_236,bert_237,bert_238,bert_239,bert_240,bert_241,bert_242,bert_243,bert_244,bert_245,bert_246,bert_247,bert_248,bert_249,bert_250,bert_251,bert_252,bert_253,bert_254,bert_255,bert_256,bert_257,bert_258,bert_259,bert_260,bert_261,bert_262,bert_263,bert_264,bert_265,bert_266,bert_267,bert_268,bert_269,bert_270,bert_271,bert_272,bert_273,bert_274,bert_275,bert_276,bert_277,bert_278,bert_279,bert_280,bert_281,bert_282,bert_283,bert_284,bert_285,bert_286,bert_287,bert_288,bert_289,bert_290,bert_291,bert_292,bert_293,bert_294,bert_295,bert_296,bert_297,bert_298,bert_299,bert_300,bert_301,bert_302,bert_303,bert_304,bert_305,bert_306,bert_307,bert_308,bert_309,bert_310,bert_311,bert_312,bert_313,bert_314,bert_315,bert_316,bert_317,bert_318,bert_319,bert_320,bert_321,bert_322,bert_323,bert_324,bert_325,bert_326,bert_327,bert_328,bert_329,bert_330,bert_331,bert_332,bert_333,bert_334,bert_335,bert_336,bert_337,bert_338,bert_339,bert_340,bert_341,bert_342,bert_343,bert_344,bert_345,bert_346,bert_347,bert_348,bert_349,bert_350,bert_351,bert_352,bert_353,bert_354,bert_355,bert_356,bert_357,bert_358,bert_359,bert_360,bert_361,bert_362,bert_363,bert_364,bert_365,bert_366,bert_367,bert_368,bert_369,bert_370,bert_371,bert_372,bert_373,bert_374,bert_375,bert_376,bert_377,bert_378,bert_379,bert_380,bert_381,bert_382,bert_383,bert_384,bert_385,bert_386,bert_387,bert_388,bert_389,bert_390,bert_391,bert_392,bert_393,bert_394,bert_395,bert_396,bert_397,bert_398,bert_399,bert_400,bert_401,bert_402,bert_403,bert_404,bert_405,bert_406,bert_407,bert_408,bert_409,bert_410,bert_411,bert_412,bert_413,bert_414,bert_415,bert_416,bert_417,bert_418,bert_419,bert_420,bert_421,bert_422,bert_423,bert_424,bert_425,bert_426,bert_427,bert_428,bert_429,bert_430,bert_431,bert_432,bert_433,bert_434,bert_435,bert_436,bert_437,bert_438,bert_439,bert_440,bert_441,bert_442,bert_443,bert_444,bert_445,bert_446,bert_447,bert_448,bert_449,bert_450,bert_451,bert_452,bert_453,bert_454,bert_455,bert_

In [14]:
bert_df_train.to_csv('pytorch_bert_train.csv', index=False)
bert_df_test.to_csv('pytorch_bert_test.csv', index=False)

### TensorFlow BERT

In [16]:
%%time

tf_tokenizer = AutoTokenizer.from_pretrained('google-bert/bert-large-uncased')
tf_model = TFAutoModel.from_pretrained('google-bert/bert-large-uncased')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

CPU times: total: 27.7 s
Wall time: 25 s


In [17]:
def get_bert_embeddings(sentences, batch_size=64):
    embeddings = []
    
    # # Calculate the number of batches
    # num_batches = len(sentences) // batch_size + (1 if len(sentences) % batch_size > 0 else 0)
    
    for i in tqdm(range(0, len(sentences), batch_size), desc="Processing batches"):
        # Create batches of sentences
        batch_sentences = sentences[i:1+batch_size]
        inputs = tf_tokenizer(batch_sentences, padding=True, truncation=True, return_tensors='tf', max_length=512)
        
        # Process each batch through the model
        outputs = tf_model(inputs)
        batch_embeddings = outputs[1]
        
        # Append embeddings (converting tensors to numpy arrays if necessary)
        embeddings.append(batch_embeddings.numpy())
    
    # Concatenate all batch embeddings
    embeddings = np.concatenate(embeddings, axis=0)
        
    return embeddings


In [18]:
%%time

tf_bert_embeddings = get_bert_embeddings(documents_train)
tf_bert_df_train = pd.DataFrame(tf_bert_embeddings)
tf_bert_df_train.columns = ['tf_bert_' + str(col) for col in tf_bert_df_train.columns]
tf_bert_df_train.head()

Processing batches:   1%|          | 2/306 [00:31<1:20:36, 15.91s/it]


IndexError: list index out of range

In [19]:
%%time

tf_bert_embeddings = get_bert_embeddings(documents_test)
tf_bert_df_test = pd.DataFrame(tf_bert_embeddings)
tf_bert_df_test.columns = ['tf_bert_' + str(col) for col in tf_bert_df_test.columns]
tf_bert_df_test.head()

Processing batches:   2%|▏         | 2/132 [00:20<21:49, 10.07s/it]


IndexError: list index out of range

In [20]:
tf_bert_df_train.to_csv('tf_bert_train.csv', index=False)
tf_bert_df_test.to_csv('tf_bert_test.csv', index=False)

NameError: name 'tf_bert_df_train' is not defined